<a href="https://colab.research.google.com/github/prajakta02/analyzing_sentiments/blob/master/Analze_sentiments_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyze Product Sentiment

In [0]:
!pip install turicreate
import turicreate

     |████████████████████████████████| 91.8MB 58kB/s 
     |████████████████████████████████| 2.1MB 45.2MB/s 
     |████████████████████████████████| 3.4MB 51.5MB/s 
     |████████████████████████████████| 86.3MB 69kB/s 
     |████████████████████████████████| 3.8MB 47.4MB/s 
     |████████████████████████████████| 450kB 55.7MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: google-auth-oauthlib 0.4.0
    Uninstalling google-auth-oauthlib-0.4.0:
      Successfully uninstalled google-auth-oauthlib-0.4.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Found existing installation: tensorflow-es

# Read product review data

In [0]:
products = turicreate.SFrame('https://d396qusza40orc.cloudfront.net/phoenixassets/amazon_baby.csv')

Downloading https://d396qusza40orc.cloudfront.net/phoenixassets/amazon_baby.csv to /var/tmp/turicreate-root/122/b139b1b8-5225-4138-9d8b-aa66bc60547a.csv

Finished parsing file https://d396qusza40orc.cloudfront.net/phoenixassets/amazon_baby.csv

Parsing completed. Parsed 100 lines in 1.17299 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105270 lines. Lines per second: 82462.8

Finished parsing file https://d396qusza40orc.cloudfront.net/phoenixassets/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 1.49614 secs.

# Explore data

In [0]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4


In [0]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [0]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [0]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3


In [0]:
len(giraffe_reviews)

785

In [0]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [0]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [0]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3,"{'and': 5.0, 'stink':1.0, 'months': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ..."


# Define what is positive and negative sentiment

In [0]:
products['rating'].show()

Materializing SArray

In [0]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [0]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [0]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [0]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [0]:
train_data,test_data = products.random_split(.8,seed=0)

In [0]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 5.000908     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 5.595380     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 6.493366     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 9.729628     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the sentiment classifier to better understand the Giraffe reviews

In [0]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [0]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [0]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [0]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5,"{'and': 1.0, 'all': 1.0,'because': 1.0, 'it': ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5,"{'and': 1.0, 'right':1.0, 'help': 1.0, 'ju ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1,"{'and': 2.0, 'all': 1.0,'being': 1.0, 'caused': ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5,"{'and': 2.0, 'all': 1.0,'love': 1.0, 'have': ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5,"{'and': 3.0, 'all': 1.0,'old': 1.0, 'almost': ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5,"{'and': 6.0, 'seven':1.0, 'already': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5,"{'and': 4.0, 'drooling':1.0, 'additionally': ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5,"{'and': 3.0, 'chew': 1.0,'sweet': 1.0, 'is': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5,"{'chew': 2.0, 'because':1.0, 'just': 2.0, 'fe ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5,"{'and': 4.0, 'everyone':1.0, 'teething': 1.0, ...",1


# Sort the Giraffe reviews according to predicted sentiment

In [0]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [0]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5,"{'giggles': 1.0, 'all':1.0, '09': 1.0, 'food': ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4,"{'all': 2.0, 'pops': 1.0,'during': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5,"{'cute': 1.0, 'all': 2.0,'just': 1.0, 'able': ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5,"{'all': 2.0, 'baby': 1.0,'before': 1.0, 'seems': ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5,"{'cute': 1.0, 'all': 1.0,'just': 2.0, 'food': ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5,"{'think': 1.0, 'they':1.0, 'just': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4,"{'cute': 1.0, 'all': 2.0,'food': 1.0, 'when': ...",1
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5,"{'just': 3.0, 'when':3.0, 'still': 1.0, 'h ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4,"{'peace': 1.0, 'month':1.0, 'bright': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4,"{'cute': 1.0, 'just':1.0, 'being': 2.0, ...",1


In [0]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,My 7 1/2 month oldgrandbaby was sittng up ...,1,"{'and': 6.0, 'this': 1.0,'now': 1.0, 'right': ...",0
Vulli Sophie the GiraffeTeether ...,I was so looking forwardto getting this for my ...,1,"{'things': 1.0, 'just':2.0, 'over': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,Received the product andsmells like cheap rub ...,1,"{'and': 1.0, 'money':2.0, 'over': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1,"{'all': 2.0, 'press':1.0, 'shape': 3.0, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2,"{'and': 2.0, 'cute': 1.0,'btw': 1.0, 'because': ...",0
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1,"{'and': 2.0, 'everyone':1.0, 'help': 1.0, 'gi ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1,"{'saying': 1.0,'chances': 1.0, 'caus ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1,"{'all': 1.0, 'just': 1.0,'over': 2.0, 'not': 3.0, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1,"{'just': 1.0, 'children':1.0, 'hmmm': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1,"{'refund': 1.0, 'enjoy':1.0, 'do': 1.0, 'taste': ...",0


## Show the most positive reviews

In [0]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [0]:
giraffe_reviews[1]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

# Most negative reivews

In [0]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [0]:
giraffe_reviews[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 